In [ ]:
#from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
#from sklearn.model_selection import train_test_split
import os
#from sklearn.model_selection import cross_val_score
import statistics
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode()

In [ ]:
train_data = pd.read_csv("exercise_train_data.csv")

In [ ]:
#dom_train_data = train_data[[not valor for valor in train_data.video_name.isna()]]
dom_train_data = train_data
#dom_train_data = train_data[[valor for valor in train_data.video_name.isna()]]

In [ ]:
print(len(dom_train_data.name.unique()))
dom_train_data.name.unique()

In [ ]:
dom_train_data.value_counts(subset="name")

In [ ]:
peaks = dom_train_data.loc[dom_train_data.type == "peak"]
peaks.columns = peaks.columns+"_peaks"
valleys = dom_train_data.loc[dom_train_data.type == "valley"]
valleys.columns = valleys.columns+"_valleys"
cross_train_data = peaks.merge(right = valleys, left_on=["video_name_peaks", "name_peaks"], right_on=["video_name_valleys","name_valleys"], how="inner")

In [ ]:
#normalización de variables peaks
cross_train_data["elbow_angles_norm_peaks"] = (cross_train_data["elbow_angles_peaks"]/180)
cross_train_data["elbow_angles2_norm_peaks"] = (cross_train_data["elbow_angles2_peaks"]/180)
cross_train_data["armpit_angles_norm_peaks"] = (cross_train_data["armpit_angles_peaks"]/180)
cross_train_data["armpit_angles2_norm_peaks"] = (cross_train_data["armpit_angles2_peaks"]/180)
cross_train_data["legs_angles_norm_peaks"] = (cross_train_data["legs_angles_peaks"]/180)
cross_train_data["legs_angles2_norm_peaks"] = (cross_train_data["legs_angles2_peaks"]/180) 
cross_train_data["back_angles_norm_peaks"] = (cross_train_data["back_angles_peaks"]/180)
cross_train_data["back_angles2_norm_peaks"] = (cross_train_data["back_angles2_peaks"]/180)

#normalización de variables valleys
cross_train_data["elbow_angles_norm_valleys"] = (cross_train_data["elbow_angles_valleys"]/180)
cross_train_data["elbow_angles2_norm_valleys"] = (cross_train_data["elbow_angles2_valleys"]/180)
cross_train_data["armpit_angles_norm_valleys"] = (cross_train_data["armpit_angles_valleys"]/180)
cross_train_data["armpit_angles2_norm_valleys"] = (cross_train_data["armpit_angles2_valleys"]/180)
cross_train_data["legs_angles_norm_valleys"] = (cross_train_data["legs_angles_valleys"]/180)
cross_train_data["legs_angles2_norm_valleys"] = (cross_train_data["legs_angles2_valleys"]/180) 
cross_train_data["back_angles_norm_valleys"] = (cross_train_data["back_angles_valleys"]/180)
cross_train_data["back_angles2_norm_valleys"] = (cross_train_data["back_angles2_valleys"]/180)


# KNN

In [ ]:
X = cross_train_data.loc[:,["elbow_angles_norm_peaks",
                            "elbow_angles2_norm_peaks",
                            "armpit_angles_norm_peaks",
                            "armpit_angles2_norm_peaks",
                            "legs_angles_norm_peaks",
                            "legs_angles2_norm_peaks",
                            "back_angles_norm_peaks",
                            "back_angles2_norm_peaks",
                            "elbow_angles_norm_valleys",
                            "elbow_angles2_norm_valleys",
                            "armpit_angles_norm_valleys",
                            "armpit_angles2_norm_valleys",
                            "legs_angles_norm_valleys",
                            "legs_angles2_norm_valleys",
                            "back_angles_norm_valleys",
                            "back_angles2_norm_valleys",
                            "slope_peaks",
                            "slope_valleys"]]
y = cross_train_data["name_peaks"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
X

In [ ]:
#num_exercises = train_data.exercise.unique().shape[0]
knn = KNeighborsClassifier()

In [ ]:
knn.fit(X_train,y_train)

In [ ]:
knn.score(X_test,y_test)

In [ ]:
cross_val_score(knn, X,y, scoring="accuracy")

In [ ]:
# Hiperparámetros compartidos por Maria:
knn = KNeighborsClassifier(algorithm= 'auto', leaf_size= 15, metric= 'manhattan', n_neighbors= 1, weights= 'uniform')
knn.fit(X = X_train, y = y_train)
print(knn.score(X_test,y_test))
print(cross_val_score(knn, X,y))


# KNN Test

In [ ]:
squat_dom = train_data[[not valor for valor in train_data.video_name.isna()]]
squat_dom = squat_dom[squat_dom.name == "squat"]

In [ ]:
squat_dom

In [ ]:
squat_dom["elbow_angles_norm"] = squat_dom["elbow_angles"]/np.max(squat_dom["elbow_angles"])
squat_dom["elbow_angles2_norm"] = squat_dom["elbow_angles2"]/np.max(squat_dom["elbow_angles2"])
squat_dom["armpit_angles_norm"] = squat_dom["armpit_angles"]/np.max(squat_dom["armpit_angles"])
squat_dom["armpit_angles2_norm"] = squat_dom["armpit_angles2"]/np.max(squat_dom["armpit_angles2"])
squat_dom["legs_angles_norm"] = squat_dom["legs_angles"]/np.max(squat_dom["legs_angles"])
squat_dom["legs_angles2_norm"] = squat_dom["legs_angles2"]/np.max(squat_dom["legs_angles2"])
squat_dom["back_angles_norm"] = squat_dom["back_angles"]/np.max(squat_dom["back_angles"])
squat_dom["back_angles2_norm"] = squat_dom["back_angles2"]/np.max(squat_dom["back_angles2"])


In [ ]:
prediction = knn.predict(squat_dom.loc[:,["back_angles_norm","back_angles2_norm","legs_angles_norm","legs_angles2_norm","armpit_angles_norm","armpit_angles2_norm","elbow_angles_norm","elbow_angles2_norm", "orientation_bool"]])

In [ ]:
prediction

In [ ]:
statistics.mode(prediction)

# Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
et = ExtraTreesClassifier()
dt.fit(X_train,y_train)
rf.fit(X_train,y_train)
et.fit(X_train,y_train)
print("dt: ",dt.score(X_test,y_test))
print("rf: ",rf.score(X_test,y_test))
print("et: ",et.score(X_test,y_test))
print("dt: ",cross_val_score(dt,X,y))
print("rf: ",cross_val_score(rf,X,y))
print("et: ",cross_val_score(et,X,y))

In [ ]:
# Parametros de Maria
dt = DecisionTreeClassifier(criterion= 'entropy', max_depth= 6, max_leaf_nodes= 58, min_samples_split= 2)
rf = RandomForestClassifier(max_depth = 5, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 300)
et = ExtraTreesClassifier(max_leaf_nodes= 33, min_samples_split= 2, n_estimators= 90)
dt.fit(X_train,y_train)
rf.fit(X_train,y_train)
et.fit(X_train,y_train)
#dt.fit(X,y)
#rf.fit(X,y)
#et.fit(X,y)
print("dt: ",dt.score(X_test,y_test))
print("rf: ",rf.score(X_test,y_test))
print("et: ",et.score(X_test,y_test))
print("dt: ",cross_val_score(dt,X,y))
print("rf: ",cross_val_score(rf,X,y))
print("et: ",cross_val_score(et,X,y))

# DNN

In [ ]:
from tensorflow import keras
from keras import regularizers

In [ ]:
X = cross_train_data.loc[:,["elbow_angles_norm_peaks",
                            "elbow_angles2_norm_peaks",
                            "armpit_angles_norm_peaks",
                            "armpit_angles2_norm_peaks",
                            "legs_angles_norm_peaks",
                            "legs_angles2_norm_peaks",
                            "back_angles_norm_peaks",
                            "back_angles2_norm_peaks",
                            "elbow_angles_norm_valleys",
                            "elbow_angles2_norm_valleys",
                            "armpit_angles_norm_valleys",
                            "armpit_angles2_norm_valleys",
                            "legs_angles_norm_valleys",
                            "legs_angles2_norm_valleys",
                            "back_angles_norm_valleys",
                            "back_angles2_norm_valleys",
                            "slope_peaks",
                            "slope_valleys"]]
y = cross_train_data["name_peaks"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [ ]:
y

In [ ]:
dnn_model = keras.models.Sequential([
                                keras.layers.Dense(512,activation='relu',input_shape=(X.shape[1],)),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dropout(.3),
                                keras.layers.Dense(256, activation='relu', activity_regularizer=regularizers.l1(1e-4)),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dropout(.2),
                                keras.layers.Dense(128, activation='relu'),               
                                keras.layers.BatchNormalization(),       
                                keras.layers.Dropout(.3),
                                keras.layers.Dense(64, activation='relu'),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(32, activation='relu'),
                                keras.layers.Dense(5, activation='softmax')
                                ]
)

In [ ]:
dnn_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['categorical_accuracy'])

# Y_conversion

In [ ]:
y_dict = []
for i in range(len(y.unique())):
    y_dict.append([y.unique()[i], i])

y_dict = dict(y_dict)

y_true = []
for i in range(len(y)):
    y_true.append([y[i], y_dict[y[i]]])
    
y_true = [y_true, keras.utils.to_categorical(pd.DataFrame(y_true)[1], dtype="int32")]

In [ ]:
y_dict

In [ ]:
y_true_not_encoded = pd.DataFrame(y_true[0])[1]
y_true_encoded = y_true[1]

In [ ]:
y_true_encoded

In [ ]:
dnn_model.summary()

# Testing

In [ ]:
y_train_true = []
for i in y_train:
    y_train_true.append([i, y_dict[i]])
    
y_train_true = [y_train_true, keras.utils.to_categorical(pd.DataFrame(y_train_true)[1], dtype="int32")]

y_test_true = []
for i in y_test:
    y_test_true.append([i, y_dict[i]])
    
y_test_true = [y_test_true, keras.utils.to_categorical(pd.DataFrame(y_test_true)[1], dtype="int32")]

In [ ]:
dnn_model.fit(X, y_true_encoded, validation_split=.2, batch_size=30, epochs=100)

In [ ]:
fig = go.Figure()
try:
    fig.add_scatter(y = dnn_model.history.history["val_sparse_categorical_accuracy"], name = "Validation")
    fig.add_scatter(y = dnn_model.history.history["sparse_categorical_accuracy"], name = "Training")
except:
    fig.add_scatter(y = dnn_model.history.history["val_categorical_accuracy"], name = "Validation")
    fig.add_scatter(y = dnn_model.history.history["categorical_accuracy"], name = "Training")
fig.show()

In [ ]:
dnn_model.save("dnn_model")

In [ ]:
keras.models.load_model("dnn_model")

In [ ]:
import pickle
pickle.dump(knn, open("knn_model.sav", 'wb'))
et.fit(X,y)
pickle.dump(et, open("et_2.sav", 'wb'))

In [ ]:
pickle.dump(y_dict, open("label_dict.sav", "wb"))